# Introduction
From memote, we observe that there are still quite some blocked reactions in the model (18.8%).(See report '2020-05-13-5063684.html'. Here I will go through the 31 dead-end metabolites. For each I will check if they are correctly dead-end, or if there is a connecting reaction missing that causes it to be so. ALternatively, it can be considered important for biomass and so we should revisit if some of these (e.g. vitamins) should be included in the biomass reaction. 

In [2]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [3]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [4]:
model_e_coli = cameo.load_model ('iML1515')

In [5]:
model_b_sub = cameo.load_model('iYO844')

thex2eACP_c is seen as a dead end as it is always on the product side of the reaction, mostly because both EAR60y and EAR60x are in the wrong direction. So I will flip the EAR60y reaction here.

In [252]:
model.reactions.EAR60y.bounds = (0,1000)

In [253]:
model.reactions.EAR60y.add_metabolites({
    model.metabolites.hexACP_c: 2,
    model.metabolites.nadp_c:2,
    model.metabolites.h_c:-2,
    model.metabolites.nadph_c:-2,
    model.metabolites.thex2eACP_c:-2
})

In [254]:
#same for anth_c metabolite, so I will flip the ANPRT reaction

In [255]:
model.reactions.ANPRT.bounds = (0,1000)

In [256]:
model.reactions.ANPRT.add_metabolites({
    model.metabolites.h_c:2,
    model.metabolites.ppi_c:2,
    model.metabolites.pran_c:2,
    model.metabolites.anth_c:-2,
    model.metabolites.prpp_c:-2
})

In [257]:
#same for drib_c. so flip the DRBK reaction
model.reactions.DRBK.bounds = (0,1000)

In [258]:
model.reactions.DRBK.add_metabolites({
    model.metabolites.get_by_id('2dr5p_c'):2,
    model.metabolites.adp_c:2,
    model.metabolites.atp_c:-2,
    model.metabolites.drib_c:-2
})

In [259]:
#same for pan4p_c, changePTPATi rct
model.reactions.PTPATi.bounds = (0,1000)

In [260]:
model.reactions.PTPATi.add_metabolites({
    model.metabolites.dpcoa_c:2,
    model.metabolites.ppi_c:2,
    model.metabolites.atp_c:-2,
    model.metabolites.pan4p_c:-2
})

ORNAT is step one of a two step reaction that will consume the 1pyr5c_c again. ORNTA is the reaction that covers both steps in one. So we can remove the ORNAT reaction and the 1py5c_c metabolite.

In [261]:
model.remove_metabolites(model.metabolites.get_by_id('1pyr5c_c'))

In [262]:
model.remove_reactions(model.reactions.ORNAT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list



In [263]:
#rename
model.reactions.ADOCBITRANS.id = 'CBIAT'

for pppi_c metabolite: we would expect it to be somewhat more connected. So, I looked into it.

PPK2 reaction: Bacillus has a pyrophosphate kinase annotated. running a tBLASTn for this found no significant results in our strain so i will not add it.

PPA2 reaction (inorganic triphosphatase): e. coli has this enzyme confrimed. Running a tBLASTn on the sequence with our genome, we find a significant hit. Therefore, I will add the PPA2 reaction to the model.

In [264]:
#add PPA2
model.add_reaction(Reaction(id='PPA2', name = 'Inorganic triphosphatase'))

In [265]:
model.reactions.PPA2.annotation = model_e_coli.reactions.PPA2.annotation

In [266]:
model.reactions.PPA2.add_metabolites({
    model.metabolites.h2o_c:-1,
    model.metabolites.pppi_c:-1,
    model.metabolites.h_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.ppi_c:1
})

BETALDHx and BETALDHy are the same reactions, the x one should be with NADH.

In [267]:
model.reactions.BETALDHx.add_metabolites({
    model.metabolites.nadp_c:1,
    model.metabolites.nad_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadh_c:1
})

Glycine betaine. In e. coli this metabolite is imported via an ABC transporter and exported via proton antiport. I will look if our genome has something similar annotated. Our stain has both the ABC transporter and the permease annotated. So I will add both here: the ABC transporter for uptake and the permease for efflux.

In [268]:
#first add glyb_e metabolite
model.add_metabolites(Metabolite(id='glyb_e', name = 'Glycine betaine', charge = 0, compartment = 'e', formula ='C5H11NO2' ))

In [269]:
model.metabolites.glyb_e.annotation = model_e_coli.metabolites.glyb_e.annotation

In [270]:
#add ABC transport reaction
model.add_reaction(Reaction(id='GLYBtabc', name = 'Glycine betaine transport via ABC transporter'))

In [271]:
model.reactions.GLYBtabc.annotations = model_e_coli.reactions.GLYBabcpp

In [272]:
model.reactions.GLYBtabc.add_metabolites({
    model.metabolites.atp_c:-1,
    model.metabolites.glyb_e:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.adp_c:1,
    model.metabolites.glyb_c:1,
    model.metabolites.h_c:1,
    model.metabolites.pi_c:1
})

In [273]:
#add permease
model.add_reaction(Reaction(id='GLYBt', name = 'Glycine betaine transport via proton antiport'))

In [274]:
model.reactions.GLYBt.annotation = model_e_coli.reactions.GLYBt3pp.annotation

In [275]:
model.reactions.GLYBt.add_metabolites({
    model.metabolites.glyb_c:-1,
    model.metabolites.h_e:-1,
    model.metabolites.glyb_e:1,
    model.metabolites.h_c:1
})

In [276]:
#add exchange
model.add_boundary(model.metabolites.glyb_e, type = 'exchange', reaction_id = 'EX_glyb_e', sbo_term = 'SBO:0000627')

Reaction identifier,EX_glyb_e
Name,Glycine betaine exchange
Memory address,0x018c67202b88
Stoichiometry,glyb_e <=> Glycine betaine <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [277]:
#resrtict to production
model.reactions.EX_glyb_e.bounds = (0,1000)

2hymeph: is missing the connection to salicin. This will be added here.

In [278]:
model.add_reaction(Reaction(id='SALGT', name = 'UDPglucose:salicyl-alcohol beta-D-glucosyltransferase'))

In [279]:
model.reactions.SALGT.annotation['sbo'] = 'SBO:0000176'
model.reactions.SALGT.annotation['kegg.reaction'] = 'R03558'
model.reactions.SALGT.annotation['ec-code'] = '2.4.1.172'

In [280]:
model.reactions.SALGT.add_metabolites({
    model.metabolites.udpg_c:-1,
    model.metabolites.get_by_id('2hymeph_c'):-1,
    model.metabolites.udp_c:1,
    model.metabolites.salcn_c:1
})

NO2 is currently a dead end metabolite. I weill revist the nitrogen metabolism to ensure this is correctly present in our model. 

In the medium we supply both NO3 and NH4 ions. Currently we only have NH4 transport and uptake possible. Our strain has a nitrate/nitrite transporter annotated and so this should be added. We will also add exchange of these compounds. However, the exchange of NO2 will to limited to production as it is not supplied in our default medium currently. 
Our strain has both the ABC nitrate/nitrate transporter annotated as well as NarK, which is passive and utilize the proton motive force to transport nitracte/nitrate. So I will add both transport options for both nitrate and nitrite.

__NO3 Transport__

In [281]:
#add extracellular met first
model.add_metabolites(Metabolite(id='no3_e', name = 'Nitrate', compartment = 'e', charge = -1 , formula = 'NO3'))

In [282]:
model.metabolites.no3_e.annotation = model.metabolites.no3_c.annotation

In [283]:
#addABC transport
model.add_reaction(Reaction(id='NO3tabc', name = 'Transport of NO3 via ABC'))

In [284]:
model.reactions.NO3tabc.annotation ['sbo'] = 'SBO:0000185'
model.reactions.NO3tabc.annotation ['ec-code'] = '7.3.2.4'

In [285]:
model.reactions.NO3tabc.add_metabolites({
    model.metabolites.atp_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.no3_e:-1,
    model.metabolites.adp_c:1,
    model.metabolites.no3_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c:1
})

In [286]:
#add Proton antiport
model.add_reaction(Reaction(id='NO3t', name = 'Transport of NO3 via Proton symport'))

In [287]:
model.reactions.NO3t.annotation ['sbo'] = 'SBO:0000185'

In [288]:
model.reactions.NO3t.add_metabolites({
    model.metabolites.no3_e:-1,
    model.metabolites.h_e:-1,
    model.metabolites.no3_c:1,
    model.metabolites.h_c:1
})

In [289]:
#add exchange, allow both consumption and production
model.add_boundary(model.metabolites.no3_e, type = 'exchange', reaction_id = 'EX_no3_e', lb = -1000, ub = 1000, sbo_term ='SBO:0000627')

Reaction identifier,EX_no3_e
Name,Nitrate exchange
Memory address,0x018c678f0808
Stoichiometry,no3_e <=> Nitrate <=>
GPR,
Lower bound,-1000
Upper bound,1000


__NO2 transport__

In [290]:
#add extracellular met first
model.add_metabolites(Metabolite(id='no2_e', name = 'Nitrite', compartment = 'e', charge = -1 , formula = 'NO2'))

In [291]:
model.metabolites.no2_e.annotation = model.metabolites.no2_c.annotation

In [292]:
#addABC transport
model.add_reaction(Reaction(id='NO2tabc', name = 'Transport of NO2 via ABC'))

In [293]:
model.reactions.NO2tabc.annotation ['sbo'] = 'SBO:0000185'
model.reactions.NO2tabc.annotation ['ec-code'] = '7.3.2.4'

In [294]:
model.reactions.NO2tabc.add_metabolites({
    model.metabolites.atp_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.no2_e:-1,
    model.metabolites.adp_c:1,
    model.metabolites.no2_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c:1
})

In [295]:
#add Proton antiport
model.add_reaction(Reaction(id='NO2t', name = 'Transport of NO2 via Proton symport'))

In [296]:
model.reactions.NO2t.annotation ['sbo'] = 'SBO:0000185'

In [297]:
model.reactions.NO2t.add_metabolites({
    model.metabolites.no2_e:-1,
    model.metabolites.h_e:-1,
    model.metabolites.no2_c:1,
    model.metabolites.h_c:1
})

In [298]:
#add exchange, allow only production
model.add_boundary(model.metabolites.no2_e, type = 'exchange', reaction_id = 'EX_no2_e', lb = 0, ub = 1000, sbo_term ='SBO:0000627')

Reaction identifier,EX_no2_e
Name,Nitrite exchange
Memory address,0x018c673d7a88
Stoichiometry,no2_e --> Nitrite -->
GPR,
Lower bound,0
Upper bound,1000


Nitrate can be properly converted into nitrite. The nitrate should be able to be converted into nitric oxide and ammonia. 

NO2R is in the wrong direction. This should be producing ammonia. Then I've fixed this and NO2 should no longer be a dead end.

In [299]:
model.reactions.NO2R.bounds = (-1000,0)

ADP-glucose should not be a dead end. Based on annotation, it should be made from D-glucose-1-phosphate, and can be assimilated into amylose. So I will make sure this is correctly present here.

In [300]:
#add GLCS1 reaction
model.add_reaction(Reaction(id='GLCS1', name = 'Starch synthase (ADPGlc)'))

In [301]:
model.reactions.GLCS1.annotation = model_b_sub.reactions.GLCS1.annotation

In [302]:
model.reactions.GLCS1.add_metabolites({
    model.metabolites.adpglc_c:-2,
    model.metabolites.adp_c:2,
    model.metabolites.starch_c:1,
    model.metabolites.h2o_c:-1,
    model.metabolites.h_c:2
})

In [303]:
model.reactions.GLCS1.check_mass_balance()

{}

In [304]:
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

Note: memote doesn't pick up metabolites which are only involved in one reaction that is reverisible (e.g. h2_c). These should also be checked! So here i write a script to highlight those metabolites. I will go through them by hand to check they should be connected some other way. 

In [306]:
orphan_mets  = ["cellb_c","cyan_c","catechol_c","2hbut_c","appl_c","dmbzid_c","cbi_c","tol_c","mthpglu_c","chol_c","uaaagglaa_c","arbt_c","xylu__L_c","gam_c","mnl_c","5mthf_c","pant__R_c","pppg9_c","tre_c","sucr_c","5fthf_c"]
dead_ends = ["adocbl_c","tcynt_c","psuri5p_c","thpglu_c","cpppg1_c","gmagg_c","dhettp_c","udpgalur_c","uacmamu_c","adgppuc_c","chlac_c","gthox_c","ftethlg_c","hemeO_c","hqn_c","mthgxl_c","sprm_c","unaga_c","3mcat_c","no2_c","23dhba_c","gtspmd_c"]
for met in model.metabolites:
    if len(met.reactions) == 1:
        if met.id in orphan_mets:
            continue
        elif met.id in dead_ends:
            continue
        else: 
            print (met.id)
    else: 
        continue

1Dgali_c
melt_c
prpncoa_c
ggl_c
5a4icam_c
h2_c
din_c
datp_c
inost_c
uaagmda_c
pmtcoa_c
hmgcoa_c
csn_c
dctp_c
dttp_c
dutp_c
cdpglyc_c
3hmp_c
arabinan_c
rr23bdo_c
actn_c
for_e


In [313]:
#formate is missing the exchange
model.add_boundary(model.metabolites.for_e,type = 'exchange', reaction_id = 'EX_for_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_for_e
Name,Formate exchange
Memory address,0x018c67f2e288
Stoichiometry,for_e --> Formate -->
GPR,
Lower bound,0
Upper bound,1000


Our model has one NADK reaction. This is responsible for converting nad to nadp, consuming an NTP in the process. We only have the reaction with ATP, but it has been shown that this enzyme can work also with dATP, dCTP, dGTP, dTTP and GTP. So I will add these reactions here. This should alleviate the dead end that the datp, dctp and dttp are involved in.

In [373]:
#add NADK1
model.add_reaction(Reaction(id='NADK1', name = 'NAD kinase GTP'))

In [374]:
model.reactions.NADK1.annotation = model_b_sub.reactions.NADK1.annotation

In [376]:
model.reactions.NADK1.add_metabolites({
    model.metabolites.gtp_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.gdp_c:1,
    model.metabolites.nadp_c:1
})

In [379]:
#add NADK2
model.add_reaction(Reaction(id='NADK2', name = 'NAD kinase dATP'))

In [380]:
model.reactions.NADK2.annotation = model_b_sub.reactions.NADK2.annotation

In [381]:
model.reactions.NADK2.add_metabolites({
    model.metabolites.datp_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.dadp_c:1,
    model.metabolites.nadp_c:1
})

In [385]:
#add NADK3
model.add_reaction(Reaction(id='NADK3', name = 'NAD kinase dGTP'))

In [386]:
model.reactions.NADK3.annotation = model_b_sub.reactions.NADK3.annotation

In [387]:
model.reactions.NADK3.add_metabolites({
    model.metabolites.dgtp_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.dgdp_c:1,
    model.metabolites.nadp_c:1
})

In [390]:
#add NADK4
model.add_reaction(Reaction(id='NADK4', name = 'NAD kinase dCTP'))

In [391]:
model.reactions.NADK4.annotation = model_b_sub.reactions.NADK4.annotation

In [392]:
model.reactions.NADK4.add_metabolites({
    model.metabolites.dctp_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.dcdp_c:1,
    model.metabolites.nadp_c:1
})

In [395]:
#add NADK5
model.add_reaction(Reaction(id='NADK5', name = 'NAD kinase dTTP'))

In [396]:
model.reactions.NADK5.annotation = model_b_sub.reactions.NADK5.annotation

In [397]:
model.reactions.NADK5.add_metabolites({
    model.metabolites.dttp_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.dtdp_c:1,
    model.metabolites.nadp_c:1
})

In [36]:
#fix rct id
model.reactions.HMGCOALY.id = 'HMGL'

In [37]:
model.reactions.HMGL.bounds = (0,1000)

There are some metabolites that should be conisdered to be added to the biomass reaction. I will look into those in a seperate notebook.

For the others, just leave them. They don't seem wrong perse but also no idea how to connect them, so they can just be a knowledge gap. Maybe are involved in carbohydrate or storage polymer generation. 

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')